# Reconstruction using inverse problem with measured patterns

In [1]:
import sys
import numpy as np
import scipy

import matplotlib.pyplot as plt

#sys.path.append('//home/crombez/Documents/PhD/python/openspyrit/spyrit/spyrit/learning/') 
from spyrit.misc.statistics import stat_mean_coef_from_model
from spyrit.misc.metrics import *
from spyrit.learning.nets import *
from spyrit.learning.model_Had_1D_DCAN import compNet_1D, Pinv_reg, compNet_1D_test_product, compNet_1D_size_im, compNet_1D_size_im_f,compNet_1D_size_stat,compNet_1D_size_stat_norma

sys.path.append('/home/crombez/Documents/PhD/python/MOPDC/fonction/') 
from data_visualisation import *
from load_data import *
from matrix_tools import *
from walsh_hadamard import *
from walsh_ordered_recon import *

In [2]:
%matplotlib 
#inline 

Using matplotlib backend: Qt5Agg


## Reconstruction parameters

In [3]:
Nl = 512#256#
Nc = 512#256#
Nlamb = 128
Nh = 128#64#

## Hadamard

### Load patterns

In [4]:
save_path = '/home/crombez/Documents/PhD/python/PTND/Data/eGFP_DsRed_3D/Reconstruction/Mat_rc/'
name = 'motifs_Hadamard_'+str(Nh)+'_'+str(Nc)+'.npy'
motifs = np.load(save_path+name)
plot_im2D(motifs)

### Reconstruction

In [5]:
Pinv = np.linalg.pinv(np.transpose(motifs/np.max(motifs)))

In [8]:
load_path = '/home/crombez/Documents/PhD/python/PTND/Data/eGFP_DsRed_3D/Preprocess/'
list_file = ['T1_RUN0002_2023_03_13_Had_512_128_128.npy','T2_RUN0003_2023_03_13_Had_512_128_128.npy','T3_RUN0004_2023_03_13_Had_512_128_128.npy','T4_RUN0005_2023_03_13_Had_512_128_128.npy','T5_RUN0006_2023_03_13_Had_512_128_128.npy','T6_RUN0001_2023_03_14_Had_512_128_128.npy','T7_RUN0002_2023_03_14_Had_512_128_128.npy','T8_RUN0003_2023_03_14_Had_512_128_128.npy','T9_RUN0004_2023_03_14_Had_512_128_128.npy','T10_RUN0005_2023_03_14_Had_512_128_128.npy','T11_RUN0006_2023_03_14_Had_512_128_128.npy','T12_RUN0007_2023_03_14_Had_512_128_128.npy','T13_RUN0008_2023_03_14_Had_512_128_128.npy','T14_RUN0009_2023_03_14_Had_512_128_128.npy','T15_RUN0010_2023_03_14_Had_512_128_128.npy','T16_RUN0011_2023_03_14_Had_512_128_128.npy','T17_RUN0012_2023_03_14_Had_512_128_128.npy','T18_RUN0013_2023_03_14_Had_512_128_128.npy','T19_RUN0014_2023_03_14_Had_512_128_128.npy','T20_RUN0015_2023_03_14_Had_512_128_128.npy','T21_RUN0016_2023_03_14_Had_512_128_128.npy','T22_RUN0017_2023_03_14_Had_512_128_128.npy','T23_RUN0018_2023_03_14_Had_512_128_128.npy','T24_RUN0019_2023_03_14_Had_512_128_128.npy','T25_RUN0020_2023_03_14_Had_512_128_128.npy','T26_RUN0021_2023_03_14_Had_512_128_128.npy']


for file in list_file:
    
    hyper_cube = np.load(load_path+file)
    rc_pinv = np.zeros((Nl,Nc,Nlamb))
    
    for i in range(Nlamb):
        rc_pinv[:,:,i] = np.dot(hyper_cube[:,:,i],Pinv)
        
    Save_path = '/home/crombez/Documents/PhD/python/PTND/Data/eGFP_DsRed_3D/Reconstruction/hyper_cube/'
    file_name = file[:-16]+'_rc_pinv_'+str(Nl)+'x'+str(Nc)+'x'+str(Nlamb)+'.npy'
    np.save(Save_path+file_name,rc_pinv)

In [9]:
stack = np.zeros((Nl,Nc,))
for i in range(Nlamb):
    stack += rc_pinv[:,:,i]
plot_im2D(stack)

## Save lambda stack

In [ ]:

Save_path = '/home/crombez/Documents/PhD/python/PTND/Data/mRFP_DsRed_can_vs_had//Reconstruction/hyper_cube/'
stack = np.zeros((Nl,Nc))
run = 'RUN0006'
rc_pinv = np.load(Save_path+run+'_rc_pinv_'+str(Nl)+'x'+str(Nc)+'x'+str(Nlamb)+'.npy')
for i in range(Nlamb):
    stack += rc_pinv[:,:,i]
plot_im2D(stack)

In [ ]:

save_path_stack = '/home/crombez/Documents/PhD/python/PTND/Data/mRFP_DsRed_can_vs_had/Analyse_out/stack/'

if not os.path.exists(save_path_stack):
    os.makedirs(save_path_stack)
file_name = run+'_rc_pinv_lambda_stack_'+str(Nl)+'x'+str(Nc)+'.npy'
np.save(save_path_stack+file_name,stack)